# Python objects


In [ ]:
from math import sqrt

def l2_norm(*dims):
    return sqrt(sum([d**2 for d in dims]))



In [ ]:
# from abc import ABC, abstractmethod

# class BasePoint(ABC):
   
#     @abstractmethod
#     def magnitude(self):
#         pass

In [ ]:
class Point:
    x: int
    y: int
        
    def __init__(self, x, y):
        self.x = x
        self.y = y
         
    def __repr__(self):
        return f'Point<x={self.x} y={self.y}>'

In [ ]:
p = Point(1,-1)

In [ ]:
p

In [ ]:
p.__dict__

In [ ]:
Point(1,-1) == p

In [ ]:
Point(1,-1) is p

In [ ]:
points = [Point(0,0), Point(-1,1), Point(1,1)]

In [ ]:
points.sort()
points

In [ ]:
p.magnitude

## Dataclasses

Dataclasses (new in 3.7) provide some "free" methods for objects that will mostly be used to hold data (state).

In [ ]:
from dataclasses import dataclass, field, asdict, InitVar

In [ ]:
@dataclass
class Point:
    x: int
    y: int 
        
    @property
    def magnitude(self):
        return l2_norm(self.x, self.y)

In [ ]:
p = Point(1,-1)

In [ ]:
p

In [ ]:
asdict(p)

In [ ]:
p.magnitude

In [ ]:
Point(1,-1) == p

In [ ]:
Point(1,-1) is p

In [ ]:
points = [Point(0,0), Point(-1,1), Point(1,1)]

In [ ]:
points.sort()
points

## Marshmallow for ser-des operations

marshmallow is an ORM/ODM/framework-agnostic library for converting complex datatypes, such as objects, to and from native Python datatypes.

In [ ]:
from marshmallow import Schema, fields, post_load

In [ ]:
class PointSchema(Schema):
    x = fields.Integer()
    y = fields.Integer()
    

In [ ]:
p = Point(1,-1)

### Marshal to dictionaries and lists of dictionaries with automatic validation (from strings)

In [ ]:
PointSchema().dump(p)

In [ ]:
PointSchema().load({
    'x':'a',
    'y':-1,
    'z': 10
})

In [ ]:
PointSchema(many=True).load([
    {'x':1, 'y':10},
    {'x':0, 'y':0}
])